<a href="https://colab.research.google.com/github/IzzulGod/GPT2-Small-Indonesian/blob/main/source/evol_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets peft accelerate
!pip install -U datasets fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("cahya/gpt2-small-indonesian-522M")
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    "cahya/gpt2-small-indonesian-522M",
    torch_dtype=torch.float16,
    device_map="auto"
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [3]:
from datasets import load_dataset

dataset = load_dataset("FreedomIntelligence/evol-instruct-indonesian")

def convert_to_simple_chat_with_eos(example):
    messages = example["conversations"]
    chat_lines = []
    for i, msg in enumerate(messages):
        text = msg["value"].strip()
        if msg["from"] == "human":
            chat_lines.append(f"User: {text}")
        else:
            is_last = (i == len(messages) - 1)
            end_token = " </s>" if is_last else ""
            chat_lines.append(f"AI: {text}{end_token}")
    return {"text": "\n".join(chat_lines)}

dataset = dataset.map(
    convert_to_simple_chat_with_eos,
    remove_columns=["conversations", "id"]
)

print(dataset["train"][0]["text"])


model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/124 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


evol-instruct-indonesian.json:   0%|          | 0.00/161M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/59022 [00:00<?, ? examples/s]

Map:   0%|          | 0/59022 [00:00<?, ? examples/s]

User: Apa kode R untuk membuat dataframe dengan tiga kolom bernama "name", "age", dan "salary"? Kolom "name" harus memiliki nilai "John", "Mary", dan "Peter". Kolom "age" harus memiliki nilai 25, 30, dan 35. Kolom "salary" harus memiliki nilai 50000, 60000, dan 70000. Mohon berikan solusi [kode R] untuk membuat dataframe ini.
AI: Untuk membuat dataframe dengan tiga kolom bernama "name", "age", dan "salary", serta dengan nilai-nilai yang telah disebutkan, dapat dilakukan dengan menggunakan kode R sebagai berikut:

```
df <- data.frame(name=c("John", "Mary", "Peter"), age=c(25, 30, 35), salary=c(50000, 60000, 70000))
```

Penjelasan kode diatas:

- `data.frame()`: membuat sebuah dataframe baru dengan argumen yang diinputkan di dalamnya.
- `name=c("John", "Mary", "Peter")`: nilai parameter `name` yang terdiri dari vektor karakter "John", "Mary", dan "Peter".
- `age=c(25, 30, 35)`: nilai parameter `age` yang terdiri dari vektor numerik 25, 30, dan 35.
- `salary=c(50000, 60000, 70000)`: nil

In [4]:
def tokenize(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=dataset["train"].column_names)

sample_text = dataset["train"][0]["text"]
tokenized_sample = tokenizer(sample_text)

print("\nTokenized IDs:")
print(tokenized_sample["input_ids"])

print("\nDecoded:")
print(tokenizer.decode(tokenized_sample["input_ids"]))


Map:   0%|          | 0/59022 [00:00<?, ? examples/s]


Tokenized IDs:
[53, 3188, 26, 13691, 4006, 382, 369, 936, 2589, 32743, 348, 1138, 9912, 1268, 338, 78, 2508, 638, 338, 5019, 638, 287, 338, 3404, 3474, 2, 31, 3518, 371, 338, 78, 2508, 2, 1238, 536, 2609, 338, 10567, 638, 338, 32579, 638, 287, 338, 18355, 526, 3518, 371, 338, 5019, 2, 1238, 536, 2609, 1896, 12, 1827, 12, 287, 4679, 14, 3518, 371, 338, 3404, 3474, 2, 1238, 536, 2609, 5052, 498, 12, 9392, 498, 12, 287, 11576, 498, 14, 318, 2119, 22053, 10211, 221, 59, 18212, 382, 61, 369, 936, 2589, 32743, 329, 14, 199, 7148, 26, 2291, 936, 2589, 32743, 348, 1138, 9912, 1268, 338, 78, 2508, 638, 338, 5019, 638, 287, 338, 3404, 3474, 638, 857, 348, 2609, 13, 8585, 288, 527, 4357, 12, 515, 1231, 348, 930, 4006, 382, 417, 1385, 26, 199, 199, 64, 64, 64, 199, 68, 70, 221, 28, 13, 2589, 14, 32743, 8, 78, 2508, 29, 67, 8, 2, 10567, 638, 338, 32579, 638, 338, 18355, 2557, 946, 69, 29, 67, 8, 3393, 12, 1827, 12, 4679, 553, 2619, 3474, 29, 67, 8, 5758, 498, 12, 9392, 498, 12, 11576, 498, 9, 9, 1

In [5]:
from peft import prepare_model_for_kbit_training, get_peft_model, LoraConfig, TaskType

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=64,
    lora_alpha=128,
    target_modules=["c_attn", "c_proj", "mlp.c_fc", "mlp.c_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 9,437,184 || all params: 133,876,992 || trainable%: 7.0491


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [6]:
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

training_args = TrainingArguments(
    output_dir="./gpt2-chatbot-lora",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    logging_steps=200,
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=2e-4,
    warmup_ratio=0.03,
    fp16=True,
    lr_scheduler_type="cosine",
    weight_decay=0.01,
    report_to="none"
)

train_dataset = tokenized_dataset["train"]

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)

trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
200,3.536200
400,2.949100
600,2.833200
800,2.758600
1000,2.704300
1200,2.658100
1400,2.638100
1600,2.610000
1800,2.573300
2000,2.538900


TrainOutput(global_step=5535, training_loss=2.5609495912464006, metrics={'train_runtime': 12575.7555, 'train_samples_per_second': 14.08, 'train_steps_per_second': 0.44, 'total_flos': 5.139926052293837e+16, 'train_loss': 2.5609495912464006, 'epoch': 3.0})

In [7]:
model.save_pretrained("/content/drive/MyDrive/gpt2-lora-adapter")
tokenizer.save_pretrained("/content/drive/MyDrive/gpt2-lora-adapter")


('/content/drive/MyDrive/gpt2-lora-adapter/tokenizer_config.json',
 '/content/drive/MyDrive/gpt2-lora-adapter/special_tokens_map.json',
 '/content/drive/MyDrive/gpt2-lora-adapter/vocab.json',
 '/content/drive/MyDrive/gpt2-lora-adapter/merges.txt',
 '/content/drive/MyDrive/gpt2-lora-adapter/added_tokens.json',
 '/content/drive/MyDrive/gpt2-lora-adapter/tokenizer.json')

In [8]:
from peft import PeftModel

if isinstance(model, PeftModel):

    merged_model = model.merge_and_unload()

    merged_model.save_pretrained("/content/drive/MyDrive/gpt2-lora-merged")
    tokenizer.save_pretrained("/content/drive/MyDrive/gpt2-lora-merged")
else:
    print("tidak ada adapter untuk digabungkan")
